# Superconductive Data Engineering Internship Technical Evaluation

In [64]:
import re
import sys
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

medicare_spending_hospital = pd.read_csv('/Users/camilledunning/Downloads/Medicare_Spending_Per_Beneficiary___Hospital.csv')
medicare_spending_state = pd.read_csv('/Users/camilledunning/Downloads/Medicare_Spending_Per_Beneficiary___State.csv')

medicare_spending_state.rename(columns={
    'Score': 'State Average'}, inplace=True)

# Merge tables to add a 'State Average' score column
medicare_df = pd.merge(
    medicare_spending_hospital, medicare_spending_state[
        ['State', 'State Average']], how='inner', on='State')
medicare_df.tail(5)

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Score,Footnote,Start Date,End Date,Location,State Average
4707,90011,MEDSTAR WASHINGTON HOSPITAL CENTER,110 IRVING STREET NW,WASHINGTON,DC,20010,THE DISTRICT,(202) 877-7000,MSPB-1,Medicare hospital spending per patient (Medica...,1.01,NaN,01/01/2018,12/31/2018,POINT (-77.012268 38.929157),1.01
4708,640001,LBJ TROPICAL MEDICAL CENTER,"FAGAALU VILLAGE, P O BOX LBJ",PAGO PAGO,AS,96799,EASTERN DISTRICT,(684) 633-4590,MSPB-1,Medicare hospital spending per patient (Medica...,Not Available,5.0,01/01/2018,12/31/2018,NaN,Not Available
4709,480002,GOV JUAN F LUIS HOSPITAL & MEDICAL CTR,"#4007 EST DIAMOND RUBY, CHRISTIANSTED",ST CROIX,VI,820,SAINT CROIX ISLAND,(340) 778-6311,MSPB-1,Medicare hospital spending per patient (Medica...,Not Available,5.0,01/01/2018,12/31/2018,NaN,Not Available
4710,480001,"ROY LESTER SCHNEIDER HOSPITAL,THE",9048 SUGAR ESTATE,ST THOMAS,VI,801,SAINT THOMAS ISLAND,(809) 776-8311,MSPB-1,Medicare hospital spending per patient (Medica...,Not Available,5.0,01/01/2018,12/31/2018,NaN,Not Available
4711,660001,COMMONWEALTH HEALTH CENTER,1 LOWER NAVY HILL ROAD (PO BOX 409CK),GARAPAN,MP,96950,SAIPAN,(670) 234-8950,MSPB-1,Medicare hospital spending per patient (Medica...,Not Available,5.0,01/01/2018,12/31/2018,NaN,Not Available


## Preliminary Data Exploration
First, I will check the distribution of states in the dataset to see if we are dealing with unbalanced data, which could be a potential obstacle in analysis.

In [65]:
fig = px.bar(
    medicare_df, 
    x=medicare_df['State'].value_counts().values, 
    y=medicare_df['State'].value_counts().index,
    labels={'x': 'Counts', 'y': 'State'},
    title='State Occurrences in Dataset',
    width=500, 
    height=1000, 
    orientation='h', 
    color_discrete_sequence=['#00CC96'])
fig.show()

Next, let's get a sense of the ranges for the scores, since we will be validating whether a column has scores that fall in the correct range. The score distribution facilities is about Gaussian.

In [66]:
fig = px.histogram(medicare_df, x='Score', title='Score Distribution')
fig.update_layout(width=500, height=400, margin=dict(l=20, r=20, b=20, t=40))
fig.show()

In [67]:
print('Min score value: ' + str(min([float(score) for score in medicare_df[medicare_df['Score'] != 'Not Available']['Score']])))
print('Max score value: ' + str(max([float(score) for score in medicare_df[medicare_df['Score'] != 'Not Available']['Score']])))

Min score value: 0.48
Max score value: 1.53


Next, do the same thing for the state averages. The range is smaller, which makes sense, as the previous Score distribution directly presents outliers.

In [68]:
fig = px.histogram(medicare_df, 
                   x='State Average', 
                   title='State Average Distribution', 
                   color_discrete_sequence=['#EF553B'])
fig.update_layout(width=500, height=400, margin=dict(l=20, r=20, b=20, t=40))
fig.show()

In [69]:
print('Min score value: ' + str(medicare_df['State Average'].min()))
print('Max score value: ' + str(medicare_df[medicare_df['State Average'] != 'Not Available']['State Average'].max()))

Min score value: 0.89
Max score value: 1.06


In [70]:
labels = ['Available', 'Not Available']
values = [len(medicare_df[medicare_df['Score'] != 'Not Available']), 
          len(medicare_df[medicare_df['Score'] == 'Not Available'])]
fig = go.Figure(
    data=[
        go.Pie(
            labels=labels, values=values
            )], 
            layout=go.Layout(
                title=go.layout.Title(
                    text='Percentange of Available Scores for Facility in Dataset'
                    )))
fig.update_layout(width=600, height=300, margin=dict(l=20, r=20, b=20, t=40))
fig.show()

In this dataset, much less of the 'State Average' data are missing.

In [71]:
values = [len(medicare_df[medicare_df['State Average'] != 'Not Available']), 
          len(medicare_df[medicare_df['State Average'] == 'Not Available'])]
fig = go.Figure(
    data=[
        go.Pie(
            labels=labels, values=values
            )], 
            layout=go.Layout(
                title=go.layout.Title(
                    text='Percentange of Available State Average Scores in Dataset'
                    )))
fig.update_traces(marker=dict(colors=['#19D3F3', '#FFA15A']))
fig.update_layout(width=600, height=300, margin=dict(l=20, r=20, b=20, t=40))
fig.show()

## Validation Steps
### Singular Function to Determine if a Column Passes or Fails
When we apply one of the assertion functions to a column, we generate a pandas Series of 'Pass' or 'Fail' values for each column entry. To ensure that an entire column passes, we want to make sure the resulting Series of P/F contains at least 80% 'Pass'.

In [72]:
def pass_or_fail_column(series):
    '''Passes or fails a column based on whether 80% of its values satisfy a requirement'''
    if len(series[series == 'Pass']) / len(series) >= 0.8: return 'Pass'
    return 'Fail'

### Checking the Formatting of the Addresses
For this dataset's addresses to be in the correct format, the first word in the address string should be an integer, and Python should be able to parse it as such. At least 80% of the addresses satisfy this, so the 'Address' column passes the test.

In [73]:
def test_address_format(address_string):
    '''Check if an address data point is in the correct format'''
    # Split the address string into words   
    split_string = address_string.split()

    # Extract the first word, which should be an integer
    int_string = split_string[0]

    # Try to parse int_string as a integer, if there is a ValueError, test fails
    try:
        int(int_string)
        return 'Pass'
    except ValueError:
        return 'Fail' 
    
# Apply this test to the whole column
pass_or_fail_column(medicare_df['Address'].apply(test_address_format))

'Pass'

### Checking the Formatting of the Phone Numbers
This assertion uses regex to check the formatting of the elements in the 'Phone Number' column. All of the phone numbers match this regex pattern, so the 'Phone Number' column passes the test.

In [74]:
def test_phone_number_format(phone_number_string):
    '''Check if a phone number data point is in the correct format'''
    if re.match('(\([0-9]{3}\) [0-9]{3}-[0-9]{4})', phone_number_string) == None:
        return 'Fail'
    return 'Pass'

# Apply this test to the whole column
pass_or_fail_column(medicare_df['Phone Number'].apply(test_phone_number_format))

'Pass'

### Checking the Formatting of the Dates

In [75]:
def test_date_format(date_string):
    '''Check if the inputted date is in correct format'''
    if type(
        pd.to_datetime(
            date_string, format='%m/%d/%Y', errors='coerce')
            ) != pd._libs.tslibs.nattype.NaTType:
        return 'Pass'
    return 'Fail'

# Apply this test to the two date columns
print('Start Date column: ' + pass_or_fail_column(
    medicare_df['Start Date'].apply(test_date_format)))
print('End Date column: ' + pass_or_fail_column(
    medicare_df['Start Date'].apply(test_date_format)))

Start Date column: Pass
End Date column: Pass


In [76]:
def test_score_range(num):
    '''Check if the score is a non-negative decimal'''
    try:
        num = float(num)
        if num >= 0:
            return 'Pass'
        return 'Fail'
    except ValueError:
        return 'Fail'

# Apply this test to the columns - columns contain 80+% non-negative decimals
print('Facility Score Column: ' + str(pass_or_fail_column(
    medicare_df['Score'].apply(test_score_range))))
print('State Average Column: ' + str(pass_or_fail_column(
    medicare_df['State Average'].apply(test_score_range))))

print()

# Check just the ranges - ignore missing data
print('Facility Score Column without Missing Data: ' + str(
    pass_or_fail_column(medicare_df[
        medicare_df['Score'] != 'Not Available']
        ['Score'].apply(test_score_range))))



Facility Score Column: Fail
State Average Column: Pass

Facility Score Column without Missing Data: Pass


In [77]:
def test_location_format(location_string):
    try:
        float(location_string)
        return 'Fail'
    except ValueError:
        if re.match(
            'POINT \((?:-[1-9]|-?[1-8][0-9]|-?90|[0-9])+(\.\d+)? (?:-[1-9]|-?[1-9][0-9]|-?1[0-7][0-9]|-?180|[0-9])+(\.\d+)?\)?$',
            location_string) == None:
            return 'Fail'
        else:
            return 'Pass'

pass_or_fail_column(medicare_df['Location'].apply(test_location_format))

'Pass'

### Check Percentage of Missing Values in Whole Dataset

In [78]:
def test_missing_values(df):
    df = df.drop(columns=['Footnote'])
    df_size = df.shape[0] * df.shape[1]
    count = 0
    for col in df.columns:
        for x in df[col]:
            if str(x) == 'nan' or x == 'Not Available':
                count += 1
            
    if count / df_size < 0.8:
        return 'Pass'
    return 'Fail'

print(test_missing_values(medicare_df))

Pass


## Analysis Ideas/Questions
1. A potential analysis of this healthcare spending dataset can address health inequities in specific populations. Compare Medicare spending in localities with low access to patient care to that of localities with high access.
2. Is a state with higher internal variability in Medicare spending more likely to spend more or less on healthcare overall?
3. Is there poor access to healthcare in the same areas where there is poor access to food (i.e., grocery store/food deserts)?
4. Examine potential bias in data collection. Maybe some states or cities have more missing scores than others.